## Python Decorators

A decorator is a function that takes another function as an argument and modifies the behavior of the passed function without explicitly changing the source code. Decorators have many different use cases but are typically used when minor changes need to be made to many different functions.

Before we delve into decorators any further let's first understand two interesting ways in which python functions can be used.
- Using Functions as First-Class Citizens: In Python, functions can be assigned to variables, passed as arguments to other functions, and returned from functions. This property makes decorators possible.

- Nested Functions: Python allows functions to be defined within other functions and a decorator is often implemented as a nested function. We will see its benefit shortly.

### A very basic example of how a decorator is written and what it does

In [35]:
# let's say we have an adder function that adds two numbers
def adder(a, b):
    result = a + b
    print(f"Result: {result}")
    return result

In [37]:
x = adder(5, 3)

Result: 8


In [39]:
print(x)

8


Now, let's say we want to print a welcome message when the function starts to execute and a closing message  when the work is done. Our first instinct would be to use a modified adder function where we use some print satements.

In [17]:
def adder_v2(a, b):
    print("Welcome. The function will now start your calculations.")
    print(f"Result: {a+b}")
    print("Success! The function ran without any Errors.")
    return a + b

In [15]:
x = adder_v2(5, 3)

Welcome. The function will now start your calculations.
Result: 8
Success! The function ran without any Errors.


In [10]:
print(x)

8


But there are some problems with this approach.
1. This becomes very inefficient when we want to add this functionality (of printing messages) to 100s of such other functions. It's not feasible to modify all of them one by one (time inefficient and also space inefficient since we have to repeat the same code over and over again). 
2. Even if we decide to modify all those functions there another problem at hand. If the code base is a large one, as is often the case, it is very likely that the functions we want to modify were written by other people who had specific tasks in mind which they wanted to achieve. So if we go and change the code of some 100s of functions, it is very likely that we will break the existing code. Even if the code base is not that large it's still true.

So let's think about some solution. One comes to mind. Since we don't want to repeat code, can't we define a function to print our message and then just call that function inside the existing functions? Let's try that.

In [18]:
def print_msg():
    print("Welcome. The function will now start your calculations.")
    print("Success! The function ran without any Errors.")

In [40]:
# now let's modify the source code
def adder_v3(a, b):
    print_msg()
    result = a + b
    print(f"Result: {result}")
    return result

In [41]:
x = adder_v3(5, 3)

Welcome. The function will now start your calculations.
Success! The function ran without any Errors.
Result: 8


In [42]:
x

8

As you might've guessed this works somewhat but not exactly how we would want it to work. The result is printed at the end not in between the messages. There are still more problems. We would have to call the message printer function inside the existing functions if we wanted to modify all those 100s of functions. Also, in this case the printer function doesn't take any argument but it will always not be the case and we would need to pass the arguments to the modifier function if there's any. So, this is not very efficient and we are back to square one.

But don't despair. We are in the right path. We just need to be a bit more creative. So, what if instead of calling the function inside the existing function we just pass the function as an argument to the existing function and then call it inside the existing function? Let's try that.

In [26]:
# since functions are first class citizens (more simply they are objects like every other thing in python is),
# we can take them as an argument to other functions, we can return them from inside of a function and also
# we can assign them to variables (that variable can then be used to access the mother function)
def func_modifier(func):
    print("Welcome. The function will now start your calculations.")
    result = func()
    print(f"Result: {result}")
    print("Success! The function ran without any Errors.")
    return result

In [27]:
# now let's see if we've been clever enough
try: 
    func_modifier(adder)
except Exception as err_msg:
    print(err_msg)

Welcome. The function will now start your calculations.
adder() missing 2 required positional arguments: 'a' and 'b'


It seems like we've gotten to the 2nd line of the func_modifier function but then it thows an error. This is because the adder function is expecting 2 arguments but we are not passing any arguments to it. So we need to pass the arguments to the adder function as well. So how can we do that?

One approach would be to pass the arguments for the adder function along with the func itself when we are calling the func_modifier() function i.e, func_modifier(adder, a, b)... something like that and then use "a" and "b" to call the adder function. 

But here's the problem with this approach. How do we know that the functions we will be modifying all have the same number of arguments? What if some functions have 2 arguments, some have 3, some have 4 and so on? We can't possibly know that. <i> Enter *args and **kwargs to the rescue. *args and **kwargs can be used to take in any number of positional and keyword arguments.</i> Let's try it out.

In [28]:
def func_modifier(func, *args, **kwargs):
    print("Welcome. The function will now start your calculations.")
    result = func(*args, **kwargs)
    print(f"Result: {result}")
    print("Success! The function ran without any Errors.")
    return result

In [29]:
func_modifier(adder, 5, 3)

Welcome. The function will now start your calculations.
Result: 8
Success! The function ran without any Errors.


8

This is almost exactly what we want but actually not quite there yet. If we wanted to assign an instance of this modified adder function to a variable so that we can use it later on in our code, we can't do that. This is because the modified adder function is not returning an instance of the modified adder function, instead it just returns a value.

In [30]:
x = func_modifier(adder, 5, 3)

Welcome. The function will now start your calculations.
Result: 8
Success! The function ran without any Errors.


In [34]:
type(x)

int

So, let's modify our code again so that it returns a function.

In [ ]:
def func_modifier(func, *args, **kwargs):
    print("Welcome. The function will now start your calculations.")
    rv = func # we just want to return the function instead of calling it directly
    print("Success! The function ran without any Errors.")
    return rv